# Self-Driving Car Engineer Nanodegree


## Project: Build a Traffic Sign Recognition Classifier

---
## Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data
DATA_FOLDER = 'data/traffic-signs-data'

training_file = DATA_FOLDER + '/train.p'
validation_file= DATA_FOLDER + '/valid.p'
testing_file = DATA_FOLDER + '/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Dataset Summary & Exploration 

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results
import numpy as np

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Visualizing the dataset

In [ ]:
### Data exploration visualization function
def display(num_row_imgs, num_col_imgs, is_grey = 'false', hspace=0.3, wspace = 0.3):
    
    fig, axs = plt.subplots(num_row_imgs,num_col_imgs, figsize=(15, 6))
    fig.subplots_adjust(hspace, wspace)
    axs = axs.ravel()

    for i in range(num_row_imgs*num_col_imgs):

        index = random.randint(0, len(X_train))
        image = X_train[index].squeeze()
        if is_grey == 'false':
            axs[i].imshow(image)
        else:
            axs[i].imshow(image, cmap = 'gray')
        axs[i].axis('off')
        axs[i].set_title(y_train[index])
        print(y_train[index], '-->', sign_names[y_train[index]])

In [ ]:
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

import csv
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sign_names = []
with open('signnames.csv') as signname_file:
    signname_reader = csv.DictReader(signname_file)
    sign_names = [row['SignName'] for row in signname_reader]


NUM_OF_COL_IMGS = 5
NUM_OF_ROW_IMGS = 2

     
display(NUM_OF_ROW_IMGS, NUM_OF_COL_IMGS)

----

## Step 2: Design and Test a Model Architecture

### Get Data Set Statistics

In [ ]:
#Data Analysis
import random
import numpy
from matplotlib import pyplot
import numpy as np

# plotting the count of each sign
def get_data_stats():

    global y_train, y_test
    y_pos = range(n_classes)
    
    
    label_list = y_train.tolist()
    label_test_list = y_test.tolist()
    
    sign_type = ([label_list.count(n_class) for n_class in range(n_classes)])
    sign_test_type = ([label_test_list.count(n_class) for n_class in range(n_classes)])
    
    plt.figure(figsize=(20,10))
    plt.bar(y_pos, sign_type, tick_label = range(43), label = 'train')
    plt.bar(y_pos, sign_test_type, label = 'test')
    plt.ylabel('Sample Count')
    plt.xlabel('Sample Class')
    plt.legend(loc='upper right')
    plt.show()

    print("minimum samples for any label in training data:", min(np.bincount(y_train)))
    print("maximum samples for any label in training data:", max(np.bincount(y_train)))
    
    print("minimum samples for any label in test data:", min(np.bincount(y_test)))
    print("maximum samples for any label in test data:", max(np.bincount(y_test)))

In [ ]:
get_data_stats()

### Utility function : Update progress

In [ ]:
#util function -> update progress bar
import sys

def update_progress(progress, final_count):
    barLength = 1 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= final_count:
        progress = final_count
        status = "Done...\r\n"
        
    
    block = int((barLength*progress/final_count)*10)
    #print('block->', block, ' barLength->', barLength, ' barlength-block->', (barLength-block), 
    #     ' progress %', int((progress/final_count)*100))
    text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), 
                                              int((progress/final_count)*100), status)
    sys.stdout.write(text)
    sys.stdout.flush()

### Data Pre-Processing

1. Grey Scale conversion
2. Data set normalization
3. Histogram Equalization
4. Data balancing
5. Brightness Augmentation
6. Random translation and rotation

In [ ]:
#data proprocessing function
from skimage import exposure


def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def transform_image(img,ang_range,shear_range,trans_range,brightness=0):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over.

    A Random uniform distribution is used to generate different parameters for transformation

    '''
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2

    # Brightness


    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)

    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))

    if brightness == 1:
      img = augment_brightness_camera_images(img)
    
    #img = pre_process_image(img)

    return img

#data augmentation
def extended_augment_limit_diff():
    print('extended augmentation')
    global y_train, X_train
    
    max_samples = max(np.bincount(y_train))
    for class_n in range(n_classes):
        
        print(class_n, ': ', end='')
        class_indices = np.where(y_train == class_n)
        n_samples = len(class_indices[0])
        if n_samples < 800:
            for i in range(int((800 - n_samples)/5)):
                
                arr = []
                new_img = X_train[class_indices[0][i % n_samples]]
                #plt.imshow(new_img)
                #plt.show()
                for index in range(5):
                    new_img = transform_image(new_img,20,10,10)
                    
                    #plt.imshow(new_img)
                    #plt.show()
                    arr.append(new_img)
                X_train = np.concatenate((X_train, arr), axis=0)
                y_train = np.concatenate((y_train, [class_n]*5), axis=0)
                

def greyscale(inp_ar):
    inp_ar = 0.299 * inp_ar[:, :, :, 0] + 0.587 * inp_ar[:, :, :, 1] + 0.114 * inp_ar[:, :, :, 2]
    return inp_ar

def normalize(inp_ar):
    inp_ar = (inp_ar / 255.).astype(np.float32)
    return inp_ar
    
def hist_equalize(inp_ar):
    for i in range(inp_ar.shape[0]):
        inp_ar[i] = exposure.equalize_adapthist(inp_ar[i])
        update_progress(i, inp_ar.shape[0])
    return inp_ar

def reshape_after_augment(inp_ar):
    inp_ar = np.reshape(inp_ar, (inp_ar.shape[0], inp_ar.shape[2], inp_ar.shape[2], 1))
    return inp_ar

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
import numpy as np
import cv2
from sklearn.utils import shuffle
from skimage import exposure

#greyscale conversion
#print(X_train.shape)
X_train = greyscale(X_train)
X_test = greyscale(X_test)
X_valid = greyscale(X_valid)
print('greyscale conversion done...')

#normalization
X_train = normalize(X_train)
X_test = normalize(X_test)
X_valid = normalize(X_valid)
print('normalization done...')


# Apply localized histogram localization
X_train = hist_equalize(X_train)
print('train histogram equalization done...')

X_test = hist_equalize(X_test)
print('test histogram equalization done...')

X_valid = hist_equalize(X_valid)
print('valid histogram equalization done...')

#data augmentation
extended_augment_limit_diff()
print('data augmentation done')


In [ ]:
def reshape_after_augment(inp_ar):
    inp_ar = np.reshape(inp_ar, (inp_ar.shape[0], inp_ar.shape[2], inp_ar.shape[2], 1))
    return inp_ar

In [ ]:
import numpy as np
print(X_train.shape)

X_train = reshape_after_augment(X_train)
X_test = reshape_after_augment(X_test)
X_valid = reshape_after_augment(X_valid)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

### Visualize Balanced/Transformed Dataset

In [ ]:
display(NUM_OF_ROW_IMGS, NUM_OF_COL_IMGS, is_grey = 'true')

### Get Data Statistics of Updated Dataset

In [ ]:
get_data_stats()

### Model Architecture



<img src="writeup-images/architecture.png" width="800" alt="Network Architecture" />

In [ ]:
#Modified LeNet Architecture

#conv1->relu->pool->dropout->conv2->relu->pool->dropout->conv3->relu->pool->dropout->fc1->rel
import tensorflow as tf
from tensorflow.contrib.layers import flatten

conv1_W = None;
conv2_W = None;
conv3_W = None
fc1_W = None;
fc2_W = None;

def LeNet(x):   
    
    global conv1_W,conv2_W, conv3_W, fc1_W, fc2_W
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
     # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 32x32x16.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 16), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(16))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME') + conv1_b
    print ('conv1-->', conv1)
    tf.add_to_collection('vars', conv1_W)
    
    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)
    print ('conv1 activation-->', conv1)
    # SOLUTION: Pooling. Input = 32x32x16. Output = 16x16x16.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print ('conv1 pooling -->', conv1)
    
    # Dropout
    conv1 = tf.nn.dropout(conv1, keep_prob_conv1)
    print ('conv1 dropout -->', conv1)
    
    # SOLUTION: Layer 2: Convolutional. Input = 16x16x16 Output = 16x16x32?
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 16, 32), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(32))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
    print ('conv2-->', conv2)
    tf.add_to_collection('vars', conv2_W)
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)
    print ('conv2 activation-->', conv2)
    
    # SOLUTION: Pooling. Input = 16x16x32. Output = 8x8x32.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print ('conv2 pooling-->', conv2)
    
    # Dropout
    conv2 = tf.nn.dropout(conv2, keep_prob_conv2)
    print ('conv2 dropout-->', conv2)
    
    # SOLUTION: Layer 3: Convolutional. Input =8x8x32 Output = 8x8x64?
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(64))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
    print ('conv3-->', conv3)
    tf.add_to_collection('vars', conv3_W)
    
    # SOLUTION: Activation.
    conv3 = tf.nn.relu(conv3)
    print ('conv3 activation-->', conv3)

    # SOLUTION: Pooling. Input = 8x8x64. Output = 4x4x64.
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print ('conv3 pooling-->', conv3)
    
    # Dropout
    conv3 = tf.nn.dropout(conv3, keep_prob_conv3)
    print ('conv3 pooling-->', conv3)

    # SOLUTION: Flatten. Input = 4x4x64. Output = 1024.
    fc0   = flatten(conv3)
    print ('flattened layer shape-->', fc0)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 1024. Output = 2048.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(1024, 84), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(84))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    print ('fc1 shape-->', fc1)
    tf.add_to_collection('vars', fc1_W)
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    print ('fc1 activation-->', fc1)
    
    #dropout
    fc1 = tf.nn.dropout(fc1, keep_prob_fc1)
    print ('fc1 dropout-->', fc1)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc1, fc2_W) + fc2_b
    print ('logits-->', logits)
    tf.add_to_collection('vars', fc2_W)
    tf.add_to_collection('vars', logits)
    
    return logits


### Train, Validate and Test the Model

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
#keep_prob = tf.placeholder(tf.float32) # probability to keep units
keep_prob_conv1 = tf.placeholder(tf.float32) # probability to keep units
keep_prob_conv2 = tf.placeholder(tf.float32) # probability to keep units
keep_prob_conv3 = tf.placeholder(tf.float32) # probability to keep units
keep_prob_fc1 = tf.placeholder(tf.float32) # probability to keep units


EPOCHS = 50
BATCH_SIZE = 32
#ARCH_NAME = '(leNet-1fc)-1-1(hst-eq)-0-4'
ARCH_NAME = '3conv-2fc - 1-1-1-5-diff-prob-relu'
#ARCH_NAME = 'leNet-1-1-1-5-different-prob'

rate = 0.0005


logits = LeNet(x)
print('logits shape->', logits.get_shape())
regularizers = tf.nn.l2_loss(conv1_W) + tf.nn.l2_loss(conv2_W) + tf.nn.l2_loss(conv3_W) + tf.nn.l2_loss(fc1_W) + tf.nn.l2_loss(fc2_W)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy + 0.00005*regularizers)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob_conv1: 1.0, keep_prob_conv2: 1.0, keep_prob_conv3: 1.0, keep_prob_fc1: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

def plot_graph(x, y, rate, BATCH_SIZE, arch_name):
    fig1 = plt.figure()
    plt.plot(x, y)
    plt.xlim(0, len(x))
    plt.ylim(0, 100)
    plt.xlabel('epochs')
    plt.ylabel('validation-accuracy')
    plt.title(arch_name + '-->Validation Accuracy - Epoch')
    fig1.savefig('results/' + arch_name + "__" + str(rate) + "_" + str(BATCH_SIZE) + ".jpeg", bbox_inches='tight')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    validation_accuracy = []
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob_conv1: 0.9, keep_prob_conv2: 0.8, keep_prob_conv3: 0.7, keep_prob_fc1: 0.5})
         
        #print('x_valid shape->', X_valid.shape, ' y_valid shape->', y_valid.shape)
        validation_accuracy.append(evaluate(X_valid, y_valid)*100)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy[i]))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")
    plot_graph(list(np.arange(1,(EPOCHS+1))), validation_accuracy, rate, BATCH_SIZE, ARCH_NAME)
   
    
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)*100
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    with open('results/result.txt', 'a') as f:
        f.write('arch-->' + ARCH_NAME + '  epochs->' + str(EPOCHS) + ' batch size-->' + str(BATCH_SIZE) + ' accuracy->' + str(test_accuracy) + '\n')


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('./lenet.meta')
    saver.restore(sess, "./lenet")
    print("Lenet Model restored...")
    Train_accuracy = evaluate(X_train, y_train)
    print("Train Accuracy = {:.3f}".format(Train_accuracy))

---

## Test a Model on New Images

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
import numpy as np
import cv2
import os

additional_data_dir = 'additional-data/'
    
additional_images = []    
for filename in os.listdir(additional_data_dir):
    if not os.path.isdir(filename):
        image = mpimg.imread(os.path.join(additional_data_dir, filename))
        print('image file', filename, ' image shape->', image.shape)
        image = cv2.resize(image, (32, 32))
        #image = np.reshape(image, (32, 32, 3))
        plt.imshow(image)
        print('image file', filename, ' image shape->', image.shape)
        additional_images.append(np.array(image))
    
additional_images = np.asarray(additional_images)
print (additional_images.shape)


#greyscale conversion
additional_images = 0.299 * additional_images[:, :, :, 0] + 0.587 * additional_images[:, :, :, 1] + 0.114 * additional_images[:, :, :, 2]
print('greyscale conversion done...')

#normalization
additional_images = (additional_images / 255.).astype(np.float32)
print('normalization done...')

# Apply localized histogram localization  
for i in range(additional_images.shape[0]):
        additional_images[i] = exposure.equalize_adapthist(additional_images[i])
        update_progress(i, additional_images.shape[0])
print('train histogram equalization done...')



additional_images = np.reshape(additional_images, (additional_images.shape[0], additional_images.shape[2], 
                                                   additional_images.shape[2], 1))


y_additional_images = [18, 1, 34, 14, 2]
y_additional_images = np.asarray(y_additional_images)

sample_frame_2 = plt.figure(figsize=(6,6))
axes = []
for i in range(0, len(additional_images)):
    axis_2 = sample_frame_2.add_subplot(2,5,i+1)
    axis_2.set_xlabel(y_additional_images[i])
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    axis_2.imshow(additional_images[i].squeeze(), cmap = 'gray')
plt.tight_layout(h_pad=0, w_pad=0)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver3 = tf.train.import_meta_graph('./lenet.meta')
    saver3.restore(sess, "./lenet")
    internet_accuracy = evaluate(additional_images, y_additional_images)
    print("Additional Image Test Accuracy = {:.3f}".format(internet_accuracy))


### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

Internet_results = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver3 = tf.train.import_meta_graph('./lenet.meta')
    saver3.restore(sess, "./lenet")
    prediction = tf.argmax(logits, 1)
    result = sess.run(prediction, feed_dict={x: additional_images, y: y_additional_images, keep_prob_conv1: 1.0, keep_prob_conv2: 1.0, keep_prob_conv3: 1.0, keep_prob_fc1: 1.0})
    Internet_results = result
    print(result)


### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

softmax = tf.nn.softmax(logits)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver3 = tf.train.import_meta_graph('./lenet.meta')
    saver3.restore(sess, "./lenet")
    probabilities = sess.run(softmax, feed_dict={x: additional_images, y: y_additional_images, keep_prob_conv1: 1.0, keep_prob_conv2: 1.0, keep_prob_conv3: 1.0, keep_prob_fc1: 1.0})
    result_top_five = sess.run(tf.nn.top_k(probabilities, k=5))
    print (result_top_five)


---

## Step : Visualize the Neural Network's State with Test Images


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input, keep_prob_conv1:1.0, keep_prob_conv2:1.0, keep_prob_conv3:1.0, keep_prob_fc1:1.0})
    featuremaps = activation.shape[3]
    print('activation', featuremaps)
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(8,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        plt.axis('off')
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max)#, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max)#, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min)#, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest")#, cmap="gray")
            


In [ ]:
additional_data_dir = 'additional-data/'
    
#additional_images = []    
for filename in os.listdir(additional_data_dir):
    if not os.path.isdir(filename):
        image = mpimg.imread(os.path.join(additional_data_dir, filename))
        print('image file', filename, ' image shape->', image.shape)
        
        plt.imshow(image)
        break

In [ ]:
l_names = ['add_0:0', 'add_1:0', 'add_2:0']

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver3 = tf.train.import_meta_graph('./lenet.meta')
    saver3.restore(sess, "./lenet")
    layer = sess.graph.get_tensor_by_name(l_names[1])
    outputFeatureMap(image_input=[additional_images[0]], tf_activation=layer)